## Objective

Let's use the subsetted features from L1 Lasso regularization and use them for OLS, which is more interpretable. Hopefully I won't be in a high variance situation.

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))

In [16]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import string
from StringIO import StringIO
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_validation import train_test_split

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2

In [5]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [6]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

../preprocessing/helper_functions.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group[pd.np.abs(group - group.median()) > stds * group.std()] = pd.np.nan
/Users/datascientist/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2454: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)


(508653, 40)

I'm removing ~1.5% of my rows.

## Choosing columns

In [7]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'TYPE', 'SubmittedPhoto', 'Property_Type', 'Source', 'neighborhood_from_zip']
cols_census = ['race_white',
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_food_stamps',
     'poverty_pop_w_ssi',
     'school',
     'school_std_dev',
     'housing',
     'housing_std_dev',
     'bedroom',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent',
     'rent_std_dev',
     'income',
     'income_std_dev']
cols_engineered = ['queue_wk', 'queue_wk_open', 'is_description']

In [8]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]

## Dummify

In [9]:
cols_to_dummify = df.dtypes[df.dtypes == object].index
cols_to_dummify

Index([u'TYPE', u'Property_Type', u'Source', u'neighborhood_from_zip',
       u'school', u'housing'],
      dtype='object')

In [10]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

Zoning is baseline 0 6
other is baseline 1 6
Twitter is baseline 2 6
West Roxbury is baseline 3 6
8_6th_grade is baseline 4 6
rent is baseline 5 6


In [11]:
df_dummified.shape

(508653, 253)

## Removing columns as per L1 results

In [12]:
col_blacklist = ['Property_Type_Address',
 'Property_Type_Intersection',
 'Source_Constituent Call',
 'SubmittedPhoto',
 'TYPE_ADA',
 'TYPE_Alert Boston',
 'TYPE_Animal Noise Disturbances',
 'TYPE_Automotive Noise Disturbance',
 'TYPE_BWSC General Request',
 'TYPE_BWSC Pothole',
 'TYPE_Big Buildings Online Request',
 'TYPE_Billing Complaint',
 'TYPE_Bridge Maintenance',
 'TYPE_CE Collection',
 'TYPE_Cemetery Maintenance Request',
 'TYPE_City/State Snow Issues',
 'TYPE_Contractor Complaints',
 'TYPE_Corporate or Community Group Service Day Clean Up',
 'TYPE_Downed Wire',
 'TYPE_Dumpster & Loading Noise Disturbances',
 'TYPE_Fire Department Request',
 'TYPE_Fire Hydrant',
 'TYPE_Fire in Food Establishment',
 'TYPE_Follow-Up',
 'TYPE_Food Alert - Confirmed',
 'TYPE_Food Alert - Unconfirmed',
 'TYPE_General Traffic Engineering Request',
 'TYPE_Ground Maintenance',
 'TYPE_HP Sign Application New',
 'TYPE_HP Sign Application Renewal',
 'TYPE_Heat/Fuel Assistance',
 'TYPE_Idea Collection',
 'TYPE_Knockdown Replacement',
 'TYPE_Loud Parties/Music/People',
 'TYPE_Mechanical',
 'TYPE_Misc. Snow Complaint',
 'TYPE_Mosquitoes (West Nile)',
 'TYPE_Municipal Parking Lot Complaints',
 'TYPE_New Tree Warrantee Inspection',
 'TYPE_News Boxes',
 'TYPE_No Utilities - Food Establishment - Electricity',
 'TYPE_No Utilities - Food Establishment - Flood',
 'TYPE_No Utilities - Food Establishment - Sewer',
 'TYPE_No Utilities - Food Establishment - Water',
 'TYPE_No Utilities Residential - Electricity',
 'TYPE_No Utilities Residential - Gas',
 'TYPE_No Utilities Residential - Water',
 'TYPE_OCR Metrolist',
 'TYPE_Occupying W/Out A Valid CO/CI',
 'TYPE_One Boston Day',
 'TYPE_PWD Graffiti',
 'TYPE_Parking Meter Repairs',
 'TYPE_Parks General Request',
 'TYPE_Pavement Marking Inspection',
 'TYPE_Phone Bank Service Inquiry',
 'TYPE_Planting',
 'TYPE_Poor Ventilation',
 'TYPE_Private Parking Lot Complaints',
 'TYPE_Public Events Noise Disturbances',
 'TYPE_Rat Bite',
 'TYPE_Rental Unit Delivery Conditions',
 'TYPE_Request for Litter Basket Installation',
 'TYPE_Roadway Flooding',
 'TYPE_Rooftop & Mechanical Disturbances',
 'TYPE_Schedule a Bulk Item Pickup SS',
 'TYPE_Senior Shoveling',
 'TYPE_Sewage/Septic Back-Up',
 'TYPE_Sidewalk Cover / Manhole',
 'TYPE_Sidewalk Repair (Make Safe)',
 'TYPE_Sign Shop WO',
 'TYPE_Snow Removal',
 'TYPE_Snow/Ice Control',
 'TYPE_Student Overcrowding',
 'TYPE_Transfer Not Completed',
 'TYPE_Undefined Noise Disturbance',
 'TYPE_Unit Pricing Wrong/Missing',
 'TYPE_Unsanitary Conditions - Employees',
 'TYPE_Unsanitary Conditions - Establishment',
 'TYPE_Unsanitary Conditions - Food',
 'TYPE_Utility Casting Repair',
 'TYPE_Valet Parking Problems',
 'TYPE_Walk-In Service Inquiry',
 'TYPE_Watermain Break',
 'TYPE_Work Hours-Loud Noise Complaints',
 'TYPE_Yardwaste Asian Longhorned Beetle Affected Area',
 'bedroom',
 'bedroom_std_dev',
 'earned_income_per_capita',
 'housing_own',
 'housing_std_dev',
 'income',
 'income_std_dev',
 'is_description',
 'neighborhood_from_zip_Allston / Brighton',
 'neighborhood_from_zip_Back Bay',
 'neighborhood_from_zip_Beacon Hill',
 'neighborhood_from_zip_Brookline',
 'neighborhood_from_zip_Charlestown',
 'neighborhood_from_zip_Chestnut Hill',
 'neighborhood_from_zip_Dorchester',
 'neighborhood_from_zip_Downtown / Financial District',
 'neighborhood_from_zip_Fenway / Kenmore / Audubon Circle / Longwood',
 'neighborhood_from_zip_Hyde Park',
 'neighborhood_from_zip_Jamaica Plain',
 'neighborhood_from_zip_Mattapan',
 'neighborhood_from_zip_Mission Hill',
 'neighborhood_from_zip_Roslindale',
 'neighborhood_from_zip_Roxbury',
 'neighborhood_from_zip_South Boston',
 'neighborhood_from_zip_South Boston / South Boston Waterfront',
 'neighborhood_from_zip_South End',
 'neighborhood_from_zip_West End',
 'poverty_pop_below_poverty_level',
 'poverty_pop_w_food_stamps',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_ssi',
 'queue_wk',
 'race_asian',
 'race_black',
 'race_hispanic',
 'race_other',
 'race_white',
 'rent',
 'rent_std_dev',
 'school_0_none',
 'school_11_9th_grade',
 'school_13_11th_grade',
 'school_14_12th_grade_no_diploma',
 'school_15_hs_diploma',
 'school_18_some_college_no_degree',
 'school_19_associates',
 'school_20_bachelors',
 'school_21_masters',
 'school_22_professional_school',
 'school_std_dev',
 'value',
 'value_std_dev']

In [13]:
df_dummified_and_filtered = df_dummified.drop(col_blacklist, axis=1)

## Running a model

In [14]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified_and_filtered.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified_and_filtered.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [18]:
df_dummified_and_filtered.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified_and_filtered.columns]

In [19]:
col_list = ' + '.join(df_dummified_and_filtered.drop('COMPLETION_HOURS_LOG_10', axis=1))

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()
est.summary()

NameError: name 'TYPEContractorsComplaint' is not defined

### Getting adjusted $R^2$ on test set

In [ ]:
est.save('../data/model_completion_time.pkl')

In [ ]:
model = sm.load('../data/model_completion_time.pkl')

In [ ]:
model.predict(X_test.head(1))

In [ ]:
est.predict(X_test.head(1))

In [ ]:
y_pred = est.predict(X_test)

In [ ]:
adjusted_r2(y_test, y_pred, num_features=X_test.shape[1])

In [ ]:
mean_squared_error(y_test, y_pred)**0.5

## Interpreting model

Which features are most associated with completion time?

In [194]:
df = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df.columns][1:]
df['coef_abs'] = pd.np.abs(df.coef)
df.head()

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
0,Intercept,1.927300,4.000000e-03,474.854,0.0,1.919 1.935,1.927300
1,queuewkopen,0.000065,7.770000e-07,83.586,0.0,6.34e-05 6.65e-05,0.000065
2,TYPEAbandonedBicycle,0.529900,2.500000e-02,20.933,0.0,0.480 0.580,0.529900
3,TYPEAbandonedBuilding,0.751700,3.800000e-02,19.727,0.0,0.677 0.826,0.751700
4,TYPEAbandonedVehicles,0.494700,1.000000e-02,49.882,0.0,0.475 0.514,0.494700


In [201]:
df.sort_values('coef_abs', ascending=False).head(10)

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
15,TYPECheckin,-4.0738,0.149,-27.340,0.0,-4.366 -3.782,4.0738
56,TYPEOCRFrontDeskInteractions,-4.0665,0.013,-306.177,0.0,-4.093 -4.041,4.0665
83,TYPEScaleNotVisible,-3.7789,0.372,-10.147,0.0,-4.509 -3.049,3.7789
12,TYPECallLog,-3.5989,0.019,-190.263,0.0,-3.636 -3.562,3.5989
86,TYPEShortMeasureGas,-3.4432,0.072,-47.749,0.0,-3.585 -3.302,3.4432
109,TYPEWaterinGasHighPriority,-2.9661,0.225,-13.207,0.0,-3.406 -2.526,2.9661
70,TYPEProductShortMeasure,-2.8167,0.199,-14.148,0.0,-3.207 -2.427,2.8167
108,TYPEWCCallLog,-2.7293,0.263,-10.364,0.0,-3.245 -2.213,2.7293
84,TYPEScanningOvercharge,-2.5426,0.132,-19.304,0.0,-2.801 -2.284,2.5426
41,TYPEItemPriceMissing,-2.5034,0.134,-18.708,0.0,-2.766 -2.241,2.5034


The features most associated with completion time are the categories, which makes intuitive sense, since there are many categories and they don't overlap with each other.

It is comforting that "Water in Gas High Priority" and "Short Measure Gas" are associated with faster completion time.

What about the non-category features?

In [197]:
df.sort_values('coef_abs', ascending=False)[~df.coef_name.str.contains('TYPE')]

/home/ec2-user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
0,Intercept,1.927300,4.000000e-03,474.854,0.0,1.919 1.935,1.927300
115,neighborhoodfromzipNorthEnd,-0.065800,9.000000e-03,-7.152,0.0,-0.084 -0.048,0.065800
112,SourceCitizensConnectApp,-0.062400,4.000000e-03,-16.005,0.0,-0.070 -0.055,0.062400
114,neighborhoodfromzipEastBoston,0.057300,5.000000e-03,10.663,0.0,0.047 0.068,0.057300
113,SourceSelfService,0.041500,4.000000e-03,11.510,0.0,0.034 0.049,0.041500
1,queuewkopen,0.000065,7.770000e-07,83.586,0.0,6.34e-05 6.65e-05,0.000065


To interpet the above coefficients, holding the other variables constant:
- if someone lives in the North End, their issue is associated with a 1 hour _decrease_ in completion time
- if someone lives in the East Boston, their issue is associated with a 1 hour _increase_ in completion time
- if someone sends an issue from the mobile app, their issue is associated with a 1 hour _decrease_ in completion time
- if someone sends an issue from the mobile app, their issue is associated with a 1 hour _increase_ in completion time

## Future steps:
- Use dataset before June 2016, as there are more issues that haven't been completed since then
- Make a separate model for each category. Then some of my engineered features are more likely to be significant, for some of the categories.